<p><strong><font size="6">WALOUS - Fusion LC classifications results</font></strong></p>

This python code implement the method developed by ANAGEO (ULB). 

Code developped on Linux Mint 18.1 (Ubuntu Xenial 16.04) and GRASS GIS 7.3.svn (r71315).

# Table of Contents

<div id="toc"></div>

The following cell is a Javascript section of code for building the Jupyter notebook's table of content.

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

# Define working environment

**Import libraries**

In [2]:
# Import libraries needed for setting parameters of operating system 
import os
import sys
import csv
import tempfile

In [3]:
## Import multiprocessing and functools libraries
import multiprocessing
from multiprocessing import Pool
from functools import partial

** Add folder with SCR provided belong to this notebook**

In [4]:
# Add local module to the path
src = os.path.abspath('../SRC')
if src not in sys.path:
    sys.path.append(src)

** Setup environment variables for TAIS DESKTOP (Linux Mint + GRASS Dev) **

Please edit the file in `../SRC/config.py`, containing the configuration parameters, according to your own computer setup. The following cell is used to run this file.



In [5]:
run ../SRC/config.py

In [6]:
print config_parameters

{'permanent_mapset': 'PERMANENT', 'locationepsg': '31370', 'outputfolder': '../../../Results', 'gisdb': '../../GRASSDATA', 'PYTHONLIB': '/usr/lib/python2.7', 'njobs': 8, 'GISBASE': '/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu'}


In [7]:
# Import functions that setup the environmental variables
import environ_variables as envi

In [8]:
# Set environmental variables
envi.setup_environmental_variables() 
# Display current environment variables of your computer
envi.print_environmental_variables()

MDMSESSION = mate 	
MANDATORY_PATH = /usr/share/gconf/mate.mandatory.path 	
MATE_DESKTOP_SESSION_ID = this-is-deprecated 	
LESSOPEN = | /usr/bin/lesspipe %s 	
MDM_LANG = fr_BE.UTF-8 	
LOGNAME = tais 	
USER = tais 	
HOME = /home/tais 	
XDG_VTNR = 8 	
PATH = /usr/local/bin:/home/tais/BIN:/home/tais/bin:/home/tais/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/bin:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/script:/home/tais/SRC/GRASS/grass_trunk/dist.x86_64-pc-linux-gnu/lib 	
CLICOLOR = 1 	
DISPLAY = :0.0 	
SSH_AGENT_PID = 2089 	
LANG = fr_BE.UTF-8 	
TERM = xterm-color 	
SHELL = /bin/bash 	
GIS_LOCK = $$ 	
XAUTHORITY = /home/tais/.Xauthority 	
SESSION_MANAGER = local/tais-HP-Z620-Workstation:@/tmp/.ICE-unix/2018,unix/tais-HP-Z620-Workstation:/tmp/.ICE-unix/2018 	
SHLVL = 1 	
QT_LINUX_ACCESSIBILITY_ALWAYS_ON = 1 	
INSIDE_CAJA_PYTHON =  	
QT_ACCESSIBILITY = 1 	
LD_LI

** GRASS GIS Python libraries **

In [9]:
# Import libraries needed to launch GRASS GIS in the jupyter notebook
import grass.script.setup as gsetup
# Import libraries needed to call GRASS using Python
import grass.script as gscript

** Other functions**

In [10]:
import grass_database

In [11]:
reload(grass_database)

<module 'grass_database' from '/media/tais/data/WALOUS/Processing/GithubRepository_WALOUS/SRC/grass_database.pyc'>

In [12]:
# Import function that check existance and create GRASS GIS database folder if needed
from grass_database import check_gisdb, check_location, check_mapset, working_mapset
# Import functions for processing time information
from processing_time import start_processing, print_processing_time
# Import function that generate a random name in the GRASS GIS environement
from random_layer_name import random_layer_name
# Import function that check and create folder
from mkdir import check_create_dir
# Import function that check if GRASS GIS add-on is installed and install it if needed
from gextension import check_install_addon
# Import function that change .csv delimiter
from ChangeCsvDelimiter import ChangeCsvDelimiter
# Import function for .gzip archive management
from gzip_management import decompress_gzip
# Import function for computation of proportion of categorical raster in zones (segments)
import compute_proportion_categorical
from compute_proportion_categorical import proportion_class
# Import function for getting list of class of a raster
from data_prep import data_prep

**Create list for temporary layers management**

In [13]:
# Create list for storing name of intermediates layers
TMP_rast = []
TMP_vect = []

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

In [40]:
def launch_mapset(timeclass):
    #Declare empty list that will contain the messages to return
    return_message = []
    # Check if the location exists and create it if not, with the CRS defined by the epsg code 
    return_message.append(check_location(config_parameters["gisdb"],timeclass,config_parameters["locationepsg"]))
    # Change the current working GRASS GIS session mapset
    return_message.append(working_mapset(config_parameters["gisdb"],timeclass,config_parameters["permanent_mapset"]))
    # Return
    return return_message

In [41]:
def import_inputdata(timeclass):
    #Declare empty list that will contain the messages to return
    return_message = []
    
    #Start GRASS session on a new mapset
    message = launch_mapset(timeclass)
    [return_message.append(a) for a in message]
    
    #Import the segmentation layer
    input_file = os.path.join(data["obia_folder"],"SEGMENTS","segs_herve_%s.tif"%timeclass)
    try:
        gscript.run_command('r.in.gdal', overwrite=True, input=input_file, output='segments')
        #gscript.run_command('r.external', overwrite=True, input=input_file, output='segments')
        return_message.append("Segmentation layer has been imported.")
    except:
        return_message.append("ERROR: Segmentation layer has not been imported. Please check for problem.")

    #Define computational region and save it as default
    gscript.run_command('g.region', flags='s', raster='segments')  
    
    #Import the OBIA classification
    input_file = os.path.join(data["obia_folder"],"RASTER","classification_herve_%s.tif"%timeclass)
    try:
        gscript.run_command('r.in.gdal', flags='r', overwrite=True, input=input_file, output='obia_clas')
        #gscript.run_command('r.external', overwrite=True, input=input_file, output='obia_clas')
        return_message.append("OBIA classification layer has been imported.")
    except:
        return_message.append("ERROR: OBIA classification layer has not been imported. Please check for problem.")

    #Import the Pixel-based classification
    input_file = data['pix_orth']
    try:
        gscript.run_command('r.in.gdal', flags='r', overwrite=True, input=input_file, output='pixel_clas')
        return_message.append("Pixel-based classification layer has been imported.")
    except:
        return_message.append("ERROR: Pixel-based classification layer has not been imported. Please check for problem.")
     
    #Import the tiles only of a specific 'timeclass'
    condition = "timeclass=%s"%timeclass
    try:
        gscript.run_command('v.in.ogr', overwrite=True, input=tile_file, output='cutline', 
                            where=condition, key='cat') 
        #gscript.run_command('v.external', overwrite=True, input=tile_file, output='timeclass_%s'%timeclass, where=condition) 
        return_message.append("Tiles with condition 'timeclass=%s' has been imported."%timeclass)
    except:
        return_message.append("ERROR: Tiles has not been imported. Please check for problem."%timeclass)

    # Get a list with unique 'cat' of cutlines tile
    cutlines_cats = list(gscript.parse_command('v.db.select', flags='c', map='cutline', columns='cat'))
            
    #Return
    return return_message, cutlines_cats

In [42]:
def GetTrainSegID(timeclass):
    input_file = data['training_points']
    try:
        # Import vector with training points
        tmp_layer = random_layer_name()
        gscript.run_command('v.in.ogr', overwrite=True, flags='r', input=input_file, output=tmp_layer, key='OBJECTID') 
        gscript.run_command('v.select', overwrite=True, ainput=tmp_layer, binput='cutline',
                            output='train_points', operator='overlap')
        gscript.run_command('g.remove', flags='f', type='vector', name=tmp_layer)
        # Add cutline_id in attribute table
        gscript.run_command('v.db.addcolumn', map='train_points', columns="cutline_id integer")        
        gscript.run_command('v.what.vect', map='train_points', column='cutline_id', 
                            query_map='cutline', query_column='cat')
        # Add seg_id in attribute table
        gscript.run_command('v.db.addcolumn', map='train_points', columns="seg_id integer")
        gscript.run_command('v.what.rast', map='train_points', raster='segments', column='seg_id')
        # Export attribute table as .csv
        output_file = os.path.join(config_parameters['outputfolder_training_sample'],'training_timeclass_%s'%timeclass)
        gscript.run_command('v.db.select', overwrite=True, map='train_points', separator='comma', file=output_file)
        return "Traing points with SegID for timeclass '%s' have been exported as csv."%timeclass
    except:
        return "ERROR: Export of training points with SegID for timeclass '%s' failed. Please check for problem."%timeclass

In [43]:
def GetObjectStat(timeclass, cutline_cat):
    #Declare empty list that will contain the messages to return
    return_message = []
    try:
        return_message.append("Working on cutline %s"%cutline_cat)
        # Extract the cutline polygon to work on
        return_message.append("--> Define computational region and MASK")
        condition = 'cat=%s'%cutline_cat
        gscript.run_command('v.extract', overwrite=True, input='cutline', where=condition, output='tmp_cutline')
        # Define computational region based on this polygon
        gscript.run_command('g.region', flags='a', vector='tmp_cutline')
        # Mask according to this polygon
        gscript.run_command('r.mask', overwrite=True, vector='tmp_cutline')
        # Copy the mask layer
        #gscript.run_command('g.copy', overwrite=True, raster='MASK,mask_copy')
        # Decompress and import attribute table of segments the current cutline
        return_message.append("--> Import attribute table from OBIA classification")
        in_path = os.path.join(data['obia_folder'],'ATTRIBUTS','segs_tile%s_stats.csv.gz'%cat)
        out_path = in_path[:-3]
        decompress_gzip(in_path, out_path)
        ChangeCsvDelimiter(out_path, '|',',') # ONLY needed for HERVE for the moment
        gscript.run_command('db.in.ogr', overwrite=True, input=out_path, output='attribute')
        # Get mode of pixel-based classification for each segments
        gscript.run_command('r.mode', overwrite=True, base='segments', cover='pixel_clas', output='pixel_clas_mode')
        # Joint attribute from OBIA classification and modal class from pixel-based classification
        return_message.append("--> Compute modal class from pixel-based classification for each object and joint in attributes")
        tmp_csv = tempfile.mkstemp()[1]+'.csv'
        gscript.run_command('r.univar', overwrite=True, flags='t', map='pixel_clas_mode', zones='segments', 
                            output=tmp_csv, separator='comma')
        gscript.run_command('db.in.ogr', overwrite=True, input=tmp_csv, output='pixel_mode')
        sql_query = "DROP TABLE IF EXISTS tmp_a"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        sql_query = "CREATE TABLE tmp_a AS SELECT a.*,b.min as pixel_mode FROM attribute AS a LEFT JOIN pixel_mode AS b ON a.cat_ = b.zone;"
        gscript.run_command('db.execute', overwrite=True, sql=sql_query)
        ### Compute percentage of each class from pixel-based classification
        # Extract list of classes of the pixel-based classification
        pixel_classes_list = data_prep("pixel_clas")[2]
        # Declare empty lists
        TMP_CSV = []
        TMP_BINARIES = []
        ## Compute proportion of each class of categorical raster (parallel processing).
        return_message.append("--> Compute proportion of each categorical class of pixel-based classification in segments")
        gscript.run_command('g.region', raster="mask_copy")  #Set the region to match the extend of the raster
        p=Pool(config_parameters['njobs']) #Create a 'pool' of processes and launch them using 'map' function
        func=partial(proportion_class,"pixel_clas", "segments") # Set fixed argument of the function
        output=p.map(func,pixel_classes_list) # Launch the processes for as many items in the list (if function with a return, the returned results are ordered thanks to 'map' function)
        p.close()
        p.join()
        temp_rasterlist,temp_csvlist=zip(*output)
        [TMP_BINARIES.append(x) for x in temp_rasterlist]  # Append the name of binary rasters to the list of temporary maps
        [TMP_CSV.append(x) for x in temp_csvlist]  # Append the paths to .csv files to the list of temporary .csv
        ## Delete temporary binary layers
        gscript.run_command('g.remove', flags='f', type='raster', name=','.joint(TMP_BINARIES))
        
        ## Multiple join to the attribute table
        for csv_file in TMP_CSV:
            table_name = table_name = os.path.split(csv_file)[-1].split(".csv")[0]
            proportion_column = csv.reader(open(csv_file)).next()[-1]
            return_message.append("----> Add proportion for class '%s'"%table_name.split("_")[-2])
            gscript.run_command('db.in.ogr', overwrite=True, input=csv_file, output=table_name)
            sql_query = "DROP TABLE IF EXISTS tmp_b;"
            gscript.run_command('db.execute', overwrite=True, sql=sql_query)
            sql_query = "CREATE TABLE tmp_b AS SELECT a.*,b.%s FROM tmp_a AS a LEFT JOIN %s AS b ON a.cat_ = b.cat_;"%(proportion_column,table_name)
            gscript.run_command('db.execute', overwrite=True, sql=sql_query)
            sql_query = "DROP TABLE IF EXISTS tmp_a;"
            gscript.run_command('db.execute', overwrite=True, sql=sql_query)
            sql_query = "CREATE TABLE tmp_a AS SELECT * FROM tmp_b;"
            gscript.run_command('db.execute', overwrite=True, sql=sql_query)

        ## Export attribute table with all features to .csv file
        output_csv = os.path.join(config_parameters['outputfolder_classfeatures'],'features_timeclass_%s_tile_%s.csv'%(timeclass,cutline_cat))
        gscript.run_command('db.select', overwrite=True, sql="SELECT * FROM tmp_a", separator='comma', output=output_csv)
        return_message.append("--> Export attribute table with feature statistics to csv")

        # Remove mask
        gscript.run_command('r.mask', flags='r')
        # Remove layers not needed anymore 
        gscript.run_command('g.remove', flags='f', type='vector', name=tmp_cutline)
        
        # Return
        return return_message
    except:
        return "ERROR: The computation of classification feature and attribute table export failed for cutline '%s' in timeclass '%s'. Please check for problem."%(cutline_cat, timeclass)

In [51]:
def delete_tmpdata(timeclass):
    #Declare empty list that will contain the messages to return
    return_message = []
       
    #Delete layers that are not needed anymore
    list_of_tmp_rast = ['pixel_clas','obia_clas','pixel_clas_mode','mask_copy','pixel_clas_cl_2','pixel_clas_cl_3','pixel_clas_cl_5','pixel_clas_cl_6','pixel_clas_cl_9','pixel_clas_cl_11','pixel_clas_cl_12','pixel_clas_cl_41','pixel_clas_cl_42']
    tmp_vect = 'train_points'

    try:
        gscript.run_command('g.remove', quiet=True, flags='f', type='raster', name=','.join(list_of_tmp_rast))
        gscript.run_command('g.remove', flags='f', type='vector', name=tmp_vect)
        return_message.append("Temporary layers have been deleted from the mapset '%s'."%timeclass)
    except:
        return_message.append("ERROR: Something went wrong during the deleting of temporary layer for the mapset '%s'. Please check."%timeclass)

    #Return
    return return_message

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

**-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-_-**

# Preprocessing

## Create general GRASSDATA

In [14]:
# Check if the GRASS GIS database exists and create it if not
check_gisdb(config_parameters["gisdb"])
# Check if the location exists and create it if not, with the CRS defined by the epsg code 
check_location(config_parameters["gisdb"],"general",config_parameters["locationepsg"])
# Change the current working GRASS GIS session mapset
working_mapset(config_parameters["gisdb"],"general",config_parameters["permanent_mapset"])

"You are now working in mapset 'general/PERMANENT'"

## Create new directories

In [15]:
# Output folder stat
config_parameters['outputfolder_training_sample'] = os.path.join(config_parameters['outputfolder'],"Training_sample")
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder_training_sample'])

The folder '../../../Results/Training_sample' already exists


In [16]:
# Output folder stat
config_parameters['outputfolder_classfeatures'] = os.path.join(config_parameters['outputfolder'],"Classification_features")
# Check and create folder if needed
check_create_dir(config_parameters['outputfolder_classfeatures'])

The folder '../../../Results/Classification_features' already exists


# Process for each test zones

## HERVE

### Get a list of 'timeclass'

In [17]:
# Import GPKG with all tiles
tile_file = os.path.join(data['obia_folder'],'tuiles_herve.gpkg')
gscript.run_command('v.in.ogr', overwrite=True, input=tile_file, output='tuiles')
gscript.run_command('g.region', vector='tuiles')
# Get a list with unique 'cutline' of in the tile layer
timeclass_list = list(gscript.parse_command('v.db.select', flags='c', map='tuiles', columns='timeclass'))
print "There are %s different 'timeclass' value (%s)"%(len(timeclass_list),','.join(timeclass_list))

There are 5 different 'timeclass' value (146555,147343,146892,146894,147324)


In [31]:
#Declare empty list for saving output messages
output_message = []

In [34]:
# Save current time
start_process = start_processing()

# Loop on each timeclass
for timeclass in timeclass_list:
    message, cutlines_cats = import_inputdata(timeclass) #Import data
    [output_message.append(a) for a in message]
    print "\n".join(message)
    message = GetTrainSegID(timeclass)
    output_message.append(message)
    print message
    # Loop on each cutline in the current timeclass
    for cat in cutlines_cats:
        message = GetObjectStat(timeclass, cat)
        [output_message.append(a) for a in message]
        print "\n".join(message)
    #Delete temporary layers
    message = delete_tmpdata(timeclass)
    output_message.append(message)
    print message
# Print processing time
print_processing_time(start_process, "Processing achieved in ")

Location '146555' already exist
You are now working in mapset '146555/PERMANENT'
Segmentation layer has been imported.
OBIA classification layer has been imported.
Pixel-based classification layer has been imported.
Tiles with condition 'timeclass=146555' has been imported.
Traing points with SegID for timeclass '146555' have been exported as csv.
Working on cutline 7286
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table 

Working on cutline 1856
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 1857
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 2753
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2699
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 154
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2457
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classifi

Working on cutline 2500
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 7155
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classif

...

Working on cutline 167
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2121
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classifi

Working on cutline 299
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 296
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classific

Working on cutline 2703
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2313
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 2250
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Location '147324' already exist
You are now working in mapset '147324/PERMANENT'
Segmentation layer has been imported.
OBIA classification layer has been imported.
Pixel-based classification layer has been imported.
Tiles with condition 'timeclass=147324' has been imported.
Traing points with SegID for timeclass '1473

Working on cutline 2752
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2501
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

## ARLON

In [ ]:
data['obia_folder'] = '../../../Data/obia_2016/ARLON'

### Get a list of 'timeclass'

In [ ]:
# Import GPKG with all tiles
tile_file = os.path.join(data['obia_folder'],'tuiles_arlon.gpkg')
gscript.run_command('v.in.ogr', overwrite=True, input=tile_file, output='tuiles')
gscript.run_command('g.region', vector='tuiles')
# Get a list with unique 'cutline' of in the tile layer
timeclass_list = list(gscript.parse_command('v.db.select', flags='c', map='tuiles', columns='timeclass'))
print "There are %s different 'timeclass' value (%s)"%(len(timeclass_list),','.join(timeclass_list))

In [31]:
#Declare empty list for saving output messages
output_message = []

In [34]:
# Save current time
start_process = start_processing()

# Loop on each timeclass
for timeclass in timeclass_list:
    message, cutlines_cats = import_inputdata(timeclass) #Import data
    [output_message.append(a) for a in message]
    print "\n".join(message)
    message = GetTrainSegID(timeclass)
    output_message.append(message)
    print message
    # Loop on each cutline in the current timeclass
    for cat in cutlines_cats:
        message = GetObjectStat(timeclass, cat)
        [output_message.append(a) for a in message]
        print "\n".join(message)
    #Delete temporary layers
    message = delete_tmpdata(timeclass)
    output_message.append(message)
    print message
# Print processing time
print_processing_time(start_process, "Processing achieved in ")

Location '146555' already exist
You are now working in mapset '146555/PERMANENT'
Segmentation layer has been imported.
OBIA classification layer has been imported.
Pixel-based classification layer has been imported.
Tiles with condition 'timeclass=146555' has been imported.
Traing points with SegID for timeclass '146555' have been exported as csv.
Working on cutline 7286
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table 

Working on cutline 1856
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 1857
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 2753
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2699
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 154
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2457
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classifi

Working on cutline 2500
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 7155
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classif

...

Working on cutline 167
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2121
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classifi

Working on cutline 299
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 296
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classific

Working on cutline 2703
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2313
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 2250
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Location '147324' already exist
You are now working in mapset '147324/PERMANENT'
Segmentation layer has been imported.
OBIA classification layer has been imported.
Pixel-based classification layer has been imported.
Tiles with condition 'timeclass=147324' has been imported.
Traing points with SegID for timeclass '1473

Working on cutline 2752
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2501
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

## CHARLEROI

In [ ]:
data['obia_folder'] = '../../../Data/obia_2016/CHARLEROI'

### Get a list of 'timeclass'

In [ ]:
# Import GPKG with all tiles
tile_file = os.path.join(data['obia_folder'],'tuiles_charleroi.gpkg')
gscript.run_command('v.in.ogr', overwrite=True, input=tile_file, output='tuiles')
gscript.run_command('g.region', vector='tuiles')
# Get a list with unique 'cutline' of in the tile layer
timeclass_list = list(gscript.parse_command('v.db.select', flags='c', map='tuiles', columns='timeclass'))
print "There are %s different 'timeclass' value (%s)"%(len(timeclass_list),','.join(timeclass_list))

In [31]:
#Declare empty list for saving output messages
output_message = []

In [34]:
# Save current time
start_process = start_processing()

# Loop on each timeclass
for timeclass in timeclass_list:
    message, cutlines_cats = import_inputdata(timeclass) #Import data
    [output_message.append(a) for a in message]
    print "\n".join(message)
    message = GetTrainSegID(timeclass)
    output_message.append(message)
    print message
    # Loop on each cutline in the current timeclass
    for cat in cutlines_cats:
        message = GetObjectStat(timeclass, cat)
        [output_message.append(a) for a in message]
        print "\n".join(message)
    #Delete temporary layers
    message = delete_tmpdata(timeclass)
    output_message.append(message)
    print message
# Print processing time
print_processing_time(start_process, "Processing achieved in ")

Location '146555' already exist
You are now working in mapset '146555/PERMANENT'
Segmentation layer has been imported.
OBIA classification layer has been imported.
Pixel-based classification layer has been imported.
Tiles with condition 'timeclass=146555' has been imported.
Traing points with SegID for timeclass '146555' have been exported as csv.
Working on cutline 7286
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table 

Working on cutline 1856
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 1857
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 2753
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2699
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 154
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2457
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classifi

Working on cutline 2500
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 7155
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classif

...

Working on cutline 167
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2121
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classifi

Working on cutline 299
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 296
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classific

Working on cutline 2703
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2313
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 2250
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Location '147324' already exist
You are now working in mapset '147324/PERMANENT'
Segmentation layer has been imported.
OBIA classification layer has been imported.
Pixel-based classification layer has been imported.
Tiles with condition 'timeclass=147324' has been imported.
Traing points with SegID for timeclass '1473

Working on cutline 2752
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2501
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

## MARCHE

In [ ]:
data['obia_folder'] = '../../../Data/obia_2016/MARCHE'

### Get a list of 'timeclass'

In [ ]:
# Import GPKG with all tiles
tile_file = os.path.join(data['obia_folder'],'tuiles_marche.gpkg')
gscript.run_command('v.in.ogr', overwrite=True, input=tile_file, output='tuiles')
gscript.run_command('g.region', vector='tuiles')
# Get a list with unique 'cutline' of in the tile layer
timeclass_list = list(gscript.parse_command('v.db.select', flags='c', map='tuiles', columns='timeclass'))
print "There are %s different 'timeclass' value (%s)"%(len(timeclass_list),','.join(timeclass_list))

In [31]:
#Declare empty list for saving output messages
output_message = []

In [34]:
# Save current time
start_process = start_processing()

# Loop on each timeclass
for timeclass in timeclass_list:
    message, cutlines_cats = import_inputdata(timeclass) #Import data
    [output_message.append(a) for a in message]
    print "\n".join(message)
    message = GetTrainSegID(timeclass)
    output_message.append(message)
    print message
    # Loop on each cutline in the current timeclass
    for cat in cutlines_cats:
        message = GetObjectStat(timeclass, cat)
        [output_message.append(a) for a in message]
        print "\n".join(message)
    #Delete temporary layers
    message = delete_tmpdata(timeclass)
    output_message.append(message)
    print message
# Print processing time
print_processing_time(start_process, "Processing achieved in ")

Location '146555' already exist
You are now working in mapset '146555/PERMANENT'
Segmentation layer has been imported.
OBIA classification layer has been imported.
Pixel-based classification layer has been imported.
Tiles with condition 'timeclass=146555' has been imported.
Traing points with SegID for timeclass '146555' have been exported as csv.
Working on cutline 7286
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table 

Working on cutline 1856
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 1857
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 2753
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2699
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 154
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2457
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classifi

Working on cutline 2500
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 7155
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classif

...

Working on cutline 167
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2121
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classifi

Working on cutline 299
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '0'
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 296
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classific

Working on cutline 2703
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2313
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

Working on cutline 2250
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Location '147324' already exist
You are now working in mapset '147324/PERMANENT'
Segmentation layer has been imported.
OBIA classification layer has been imported.
Pixel-based classification layer has been imported.
Tiles with condition 'timeclass=147324' has been imported.
Traing points with SegID for timeclass '1473

Working on cutline 2752
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add proportion for class '2'
----> Add proportion for class '3'
----> Add proportion for class '5'
----> Add proportion for class '6'
----> Add proportion for class '9'
----> Add proportion for class '11'
----> Add proportion for class '12'
----> Add proportion for class '41'
----> Add proportion for class '42'
--> Export attribute table with feature statistics to csv
Working on cutline 2501
--> Define computational region and MASK
--> Import attribute table from OBIA classification
--> Compute modal class from pixel-based classification for each object and joint in attributes
--> Compute proportion of each categorical class of pixel-based classification in segments
----> Add propo

## Get a list of tiles id from the file name

This list will be use in a further step of developement for distribute jobs on different nodes on HYDRA. 

In [ ]:
import glob  # Library needed to get the list of file in a folder
# Will return a list of path to each file with extension '*.gpkg.gz' in the folder 
files_paths = glob.glob(os.path.join(data['obia_folder'],'VECTEUR','*.gpkg.gz'))  

list_tile_id = []
for a in files_paths:
    file_name = os.path.split(a)[1]  # Keep only the file name with extension 
    list_tile_id.append(file_name.split('_')[1][len('tile'):])  # tileid should be on index 1 in this list

print "Obia results (vector) for %s tiles found in '%s'"%(len(list_tile_id),os.path.join(data['obia_folder'],'VECTEUR'))

## Launch GRASS GIS session that will work on the first tile

In [ ]:
# Define the id of the current tile
tile_id = list_tile_id[0]
# Define the name of the location
grasslocation = "WALOUS_%s"%tile_id

In [ ]:
# Check if the GRASS GIS database exists and create it if not
check_gisdb(config_parameters["gisdb"])
# Check if the location exists and create it if not, with the CRS defined by the epsg code 
check_location(config_parameters["gisdb"],grasslocation,config_parameters["locationepsg"])
# Change the current working GRASS GIS session mapset
working_mapset(config_parameters["gisdb"],grasslocation,config_parameters["permanent_mapset"])

## Import tile polygon and define computation region

**Unzip the geopackage file**

In [ ]:
files_paths[0]

# Import data / Preparation of data

In [ ]:
# Saving current time for processing time management
begintime_importdata=start_processing()

In [ ]:
import zipfile
path = os.path.join(data['obia_folder'],'VECTEUR','segs_tile%s_stats.gpkg.gz'%tile_id)

In [ ]:
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall(directory_to_extract_to)
zip_ref.close()

In [ ]:
from sh import gunzip
gunzip(path)


## Import training samples (points)

In [ ]:
# Import point samples (only for the computational region)
#gscript.run_command('v.import', overwrite=True, input=data['admin'][1], output=data['admin'][0])

In [ ]:
## Print processing time
print_processing_time(begintime_processing ,"All processing terminate in ")

## Import pixel-based classification

## Import object-based classification

# Exports and cleaning mapset

## Export results

In [ ]:
# Output folder isochrones
outputfolder_isochrones = os.path.join(config_parameters['outputfolder'],"Isochrones")
# Check and create folder if needed
check_create_dir(outputfolder_isochrones)

In [ ]:
# Export isochrone layers as GeoPackage and attribute table as .csv
for isochrone in isochrone_layers:
    output_gpkg = os.path.join(outputfolder_isochrones,"%s.gpkg"%isochrone)
    output_csv = os.path.join(config_parameters['outputfolder'],"Prop_by_Isochrones_%s.csv"%isochrone[11:])
    gscript.run_command('v.out.ogr', flags='m', overwrite=True,
                        input=isochrone, output=output_gpkg, format="GPKG")
    gscript.run_command('v.out.ogr', flags='m', overwrite=True,
                        input=isochrone, output=output_csv, format="CSV")

## Remove intermediate layers

In [ ]:
# Remove all layers not needed anymore
for rast in TMP_rast:
    gscript.run_command('g.remove', flags='fb', type='raster', name=rast)

In [ ]:
# Remove all layers not needed anymore
for vect in TMP_vect:
    gscript.run_command('g.remove', flags='fb', type='vector', name=vect)

## Delete both mapsets

In [ ]:
import shutil

# Delete mapset folder in GRASSDATA
shutil.rmtree(config_parameters['gisdb'])